In [1]:
import os
import csv
import sys
sys.path.append('..')
import json
import random

In [2]:
from matplotlib import pyplot as plt
import pandas as pd

In [3]:
from constants import kinetics_data_path

In [4]:
an_train = pd.read_csv('./ActivityNet/Crawler/Kinetics/data/kinetics-400_train.csv')
an_test  = pd.read_csv('./ActivityNet/Crawler/Kinetics/data/kinetics-400_test.csv')
an_val   = pd.read_csv('./ActivityNet/Crawler/Kinetics/data/kinetics-400_val.csv')

In [5]:
labels = list(an_train['label'].unique())

In [6]:
# pick 5 labels
random.seed(22)
chosen_labels = random.sample(labels, 2)

In [7]:
an_test.head()

,label,youtube_id,time_start,time_end,split
0,drinking beer,--6bJUbfpnQ,17,27,test
1,climbing tree,--8YXc8iCt8,2,12,test
2,surfing water,--coBvtS-eQ,57,67,test
3,stomping grapes,--q6ElFyVq0,148,158,test
4,tai chi,--q_mvQ8zP8,67,77,test


In [8]:
examples_train = an_train[an_train['label'].isin(chosen_labels)]
examples_test = an_test[an_test['label'].isin(chosen_labels)]
examples_val = an_val[an_val['label'].isin(chosen_labels)]
examples_train = examples_train.loc[:,['youtube_id', 'time_start', 'time_end', 'label']]
examples_test = examples_test.loc[:,['youtube_id', 'time_start', 'time_end', 'label']]
examples_val = examples_val.loc[:,['youtube_id', 'time_start', 'time_end', 'label']]

In [9]:
examples_test.head()

,youtube_id,time_start,time_end,label
95,-BfA5d412Fc,21,31,feeding fish
166,-Kq1xajBxPw,350,360,cooking egg
198,-PhpwXdflY4,95,105,cooking egg
442,-tutrdjNtoU,194,204,cooking egg
584,0BCtCUXWuEs,80,90,cooking egg


In [10]:
len(examples_train), len(examples_test), len(examples_val)

(1293, 172, 99)

In [24]:
examples_train.to_csv('my_kinetics_train.csv', index=False)
examples_test.to_csv('my_kinetics_test.csv', index=False)
examples_val.to_csv('my_kinetics_val.csv', index=False)

In [29]:
!python3 ./ActivityNet/Crawler/Kinetics/download.py ./my_kinetics.csv $RAW_DATA_PATH/kinetics/data --num-jobs=10

Traceback (most recent call last):
  File "./ActivityNet/Crawler/Kinetics/download.py", line 221, in <module>
    main(**vars(p.parse_args()))
  File "./ActivityNet/Crawler/Kinetics/download.py", line 201, in main
    fobj.write(json.dumps(status_lst))
  File "/usr/lib/python3.8/json/__init__.py", line 231, in dumps
    return _default_encoder.encode(obj)
  File "/usr/lib/python3.8/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "/usr/lib/python3.8/json/encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "/usr/lib/python3.8/json/encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type bytes is not JSON serializable


In [25]:
!python3 ./ActivityNet/Crawler/Kinetics/download.py ./my_kinetics_test.csv $RAW_DATA_PATH/kinetics/data --num-jobs=10

Traceback (most recent call last):
  File "./ActivityNet/Crawler/Kinetics/download.py", line 221, in <module>
    main(**vars(p.parse_args()))
  File "./ActivityNet/Crawler/Kinetics/download.py", line 201, in main
    fobj.write(json.dumps(status_lst))
  File "/usr/lib/python3.8/json/__init__.py", line 231, in dumps
    return _default_encoder.encode(obj)
  File "/usr/lib/python3.8/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "/usr/lib/python3.8/json/encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "/usr/lib/python3.8/json/encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type bytes is not JSON serializable


In [26]:
!python3 ./ActivityNet/Crawler/Kinetics/download.py ./my_kinetics_val.csv $RAW_DATA_PATH/kinetics/data --num-jobs=10

Traceback (most recent call last):
  File "./ActivityNet/Crawler/Kinetics/download.py", line 221, in <module>
    main(**vars(p.parse_args()))
  File "./ActivityNet/Crawler/Kinetics/download.py", line 201, in main
    fobj.write(json.dumps(status_lst))
  File "/usr/lib/python3.8/json/__init__.py", line 231, in dumps
    return _default_encoder.encode(obj)
  File "/usr/lib/python3.8/json/encoder.py", line 199, in encode
    chunks = self.iterencode(o, _one_shot=True)
  File "/usr/lib/python3.8/json/encoder.py", line 257, in iterencode
    return _iterencode(o, 0)
  File "/usr/lib/python3.8/json/encoder.py", line 179, in default
    raise TypeError(f'Object of type {o.__class__.__name__} '
TypeError: Object of type bytes is not JSON serializable


In [11]:
# prepare the files with video paths and labels
train_files = []
test_files = []
val_files = []
notfound = []
directory = f'{os.environ["RAW_DATA_PATH"]}/kinetics/data'
class_dirs = [d for d in os.listdir(directory) if os.path.isdir(os.path.join(directory,d))]
for i, folder_name in enumerate(class_dirs):
    folder_path = os.path.join(directory, folder_name)
    for file_name in  os.listdir(folder_path):
        file_path = os.path.join(folder_path, file_name)
        
        ytid = file_name[:-18]

        if ytid in examples_train['youtube_id'].to_list():
            train_files.append((file_path, i))
        elif ytid in examples_test['youtube_id'].to_list():
            test_files.append((file_path, i))
        elif ytid in examples_val['youtube_id'].to_list():
            val_files.append((file_path, i))
        else:
            notfound.append(ytid)

In [13]:
pd.DataFrame(train_files).to_csv(os.path.join(os.environ["RAW_DATA_PATH"], 'kinetics', 'data', 'train.csv'), index=False, header=False, sep=' ')
pd.DataFrame(test_files).to_csv(os.path.join(os.environ["RAW_DATA_PATH"], 'kinetics', 'data', 'test.csv'), index=False, header=False, sep=' ')
pd.DataFrame(val_files).to_csv(os.path.join(os.environ["RAW_DATA_PATH"], 'kinetics', 'data', 'val.csv'), index=False, header=False, sep=' ')

In [ ]:
# https://github.com/facebookresearch/SlowFast/issues/149#issuecomment-723265461

# https://pytorchvideo.org/docs/tutorial_classification